In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['JAVA_HOME'] = '/Users/sb0487/Desktop/sb0487/pyspark/zulu15.32.15-ca-jdk15.0.3-macosx_aarch64/zulu-15.jdk/Contents/Home' 

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("ggplot")
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
import cufflinks as cf
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import pandas as pd
import numpy as np
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from ipywidgets import interact, interact_manual
from pyspark.sql.window import Window
import pendulum as plm
from pyspark.sql.types import IntegerType
from pyspark.sql import DataFrame
import pendulum
from datetime import datetime
cf.go_offline(connected=True)
init_notebook_mode(connected=True)

In [3]:
from pyspark.sql import SparkSession as Session
from pyspark import SparkConf as Conf
from pyspark.sql import functions as F, Window as W
C = F.col

In [4]:
conf = (Conf()
        .set("spark.sql.sources.partitionOverwriteMode", "dynamic")
        .set("spark.driver.memory", "3g")
        .set("spark.sql.broadcastTimeout", "10200")
        .set("spark.driver.maxResultSize", "1g")
#         .set("spark.local.dir","/buffer") 指定shuffle write的路徑
       ) 

In [5]:
spark = (Session
         .builder
         .appName("tradddddd")
         .master("local[4]")# 核心數量
         .config(conf=conf)
         .getOrCreate())

In [6]:
spark

# 說明

將台指資料做特徵擴展,並用ML預測漲跌，觀察是否能提升準確度

# load data

In [7]:
sdf = spark.read.csv("/Users/sb0487/Desktop/sb0487/finance/quant_project/data/TXF_investor_chip_no_adjust.csv",header = True)


In [8]:
# load my package
import sys
sys.path.append("//Users/sb0487/Desktop/sb0487/finance/quant_project")
from func_utils.utils import format_columns_name, format_value,extend_feature,get_return_label

# format data

In [9]:
# 將日期，收盤價等轉為英文
formatted_columns_name_sdf = format_columns_name(sdf)
# 將原始資料中，"口"去除，並轉為int tpye
formatted_value_sdf = format_value(formatted_columns_name_sdf,replace_char="口")

## feature extension

In [10]:
# 擴展特徵，目前只支援一階差分，未來可能可以加入sma等，使其可以擁有型態資訊
extended_feature_sdf = extend_feature(formatted_value_sdf,lag1=True)
extended_feature_df = extended_feature_sdf.toPandas()

In [11]:
# 檢查有無缺失值
extended_feature_df.isnull().any()

Date           False
Open           False
High           False
Low            False
Close          False
外資及陸資交易買口      False
外資及陸資交易賣口      False
外資及陸資交易淨口      False
投信交易買口         False
投信交易賣口         False
投信交易淨口         False
自營商交易買口        False
自營商交易賣口        False
自營商交易淨口        False
外資及陸資未平倉買口     False
外資及陸資未平倉賣口     False
外資及陸資未平倉淨口     False
投信未平倉買口        False
投信未平倉賣口        False
投信未平倉淨口        False
自營商未平倉買口       False
自營商未平倉賣口       False
自營商未平倉淨口       False
前十大法人買口        False
前十大法人賣口        False
前十大特定法人買口      False
前十大特定法人賣口      False
前五大法人買口        False
前五大法人賣口        False
前五大特定法人買口      False
前五大特定法人賣口      False
∆外資及陸資交易買口     False
∆外資及陸資交易賣口     False
∆外資及陸資交易淨口     False
∆投信交易買口        False
∆投信交易賣口        False
∆投信交易淨口        False
∆自營商交易買口       False
∆自營商交易賣口       False
∆自營商交易淨口       False
∆外資及陸資未平倉買口    False
∆外資及陸資未平倉賣口    False
∆外資及陸資未平倉淨口    False
∆投信未平倉買口       False
∆投信未平倉賣口       False
∆投信未平倉淨口       False
∆自營商未平倉買口      False
∆自營商未平倉賣口    

## change close price to label

若採用數值預測,在損失函數為MSE情況下,可能會有以下情形

1.模型a預測今天上漲200點,實際上漲500點-> loss為300平方點

2.模型a預測明天上漲50點,實際下跌50點-> loss為100平方點

基於min sum of loss此時模型會調整權重使得模型使預測值再更高,可能會使預測變成如下

1.模型a預測今天上漲300點,實際上漲500點-> loss為200平方點

2.模型a預測明天上漲150點,實際下跌50點-> loss為20平方點

然而實際我們需要的是方向正確即可,明天下跌,也就是其實應該調整權重使預測降低,也就是模型a可預測出今天上漲,明天下跌

於是,對y做簡單的非線性轉換,改為二元分類,好處有2

1.一般金融數列多呈現I(1)

2.不用改損失函數


In [12]:
day_return_label_df = get_return_label(extended_feature_sdf).toPandas()
day_return_label_df.head(5)

,Date,Open,High,Low,Close,外資及陸資交易買口,外資及陸資交易賣口,外資及陸資交易淨口,投信交易買口,投信交易賣口,...,∆前十大特定法人買口,∆前十大特定法人賣口,∆前五大法人買口,∆前五大法人賣口,∆前五大特定法人買口,∆前五大特定法人賣口,last_day_return,last_day_return_label,day_return,day_return_label
0,2010-01-05,8212,8260,8125,8178,6648,6982,-334,757,821,...,1870,2040,1033,357,570,357,NaN,0,-34,0
1,2010-01-06,8197,8336,8179,8321,9592,7982,1610,479,353,...,2582,-1342,2346,147,535,-1405,143.0,1,124,1
2,2010-01-07,8335,8355,8230,8260,9757,17049,-7292,391,298,...,-3988,2961,174,888,-1151,2440,-61.0,0,-75,0
3,2010-01-08,8287,8297,8183,8277,5954,7364,-1410,495,562,...,-18,1589,722,779,-10,-1168,17.0,1,-10,0
4,2010-01-11,8278,8353,8251,8322,7785,5325,2460,677,533,...,2208,917,574,571,326,2518,45.0,1,44,1


## shift Y label

因為是要透過本期的X來預測下一期的Y,所以必須將下一期的漲跌label移至本期當成標籤

In [13]:
def shift_label(df):
    #shift
    df["day_return_label_shift"] = df["day_return_label"].shift(-1)
    # 因做shift,要把空值的row去除
    shift_label_df = df.dropna()
    return shift_label_df

In [14]:
shift_label_df = shift_label(day_return_label_df)

# split train test data

取2015/01~2017/12   資料訓練模型

取2018/01~/2021/06  回測

In [15]:
from pandas import Series, DataFrame
shift_train_df = shift_label_df[(shift_label_df['Date'] >= '2015-01-01') & (shift_label_df['Date'] <= '2017-12-31')]
shift_test_df = shift_label_df[(shift_label_df['Date'] >= '2018-01-01') & (shift_label_df['Date'] <= '2021-06-31')]


# 檢定


由於欄位很多,首先要選出有效因子

這邊用[grangercausalitytests](https://www.statsmodels.org/stable/generated/statsmodels.tsa.stattools.grangercausalitytests.html)

ps: 顯然淨口(存量)一定是非定態,但直覺認為這是一個重要變因,先保留

In [16]:
def do_grangercausalitytests(df, significant_level = 0.05)->list:
    from statsmodels.tsa.stattools  import   grangercausalitytests
    # 逐一檢定後, 將有效欄位選出來
    significant_col_list = []
    for i in range(1,len(df.columns)-1):
        col = df.columns[i]
        pvalue = grangercausalitytests(df[['day_return_label',col]], maxlag=1, addconst=True, verbose=True)[1][0].get("ssr_ftest")[1]
        if pvalue<=significant_level:
            print("欄位[",col,"]的pvalue為",pvalue,"拒絕虛無")
            significant_col_list.append(col)
        else:
            print("欄位[",col,"]的pvalue為",pvalue,"接受虛無")
        #加回日期,收盤價等  
    significant_col_list.extend(["Close","Open","last_day_return","last_day_return_label","day_return","day_return_label_shift","Date"])
    return significant_col_list
    

In [17]:
significant_col_list = do_grangercausalitytests(shift_train_df,0.05)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.2759  , p=0.2590  , df_denom=730, df_num=1
ssr based chi2 test:   chi2=1.2812  , p=0.2577  , df=1
likelihood ratio test: chi2=1.2801  , p=0.2579  , df=1
parameter F test:         F=1.2759  , p=0.2590  , df_denom=730, df_num=1
欄位[ Open ]的pvalue為 0.2590249004838815 接受虛無

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.1613  , p=0.2816  , df_denom=730, df_num=1
ssr based chi2 test:   chi2=1.1660  , p=0.2802  , df=1
likelihood ratio test: chi2=1.1651  , p=0.2804  , df=1
parameter F test:         F=1.1613  , p=0.2816  , df_denom=730, df_num=1
欄位[ High ]的pvalue為 0.28155963467591383 接受虛無

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.4900  , p=0.2226  , df_denom=730, df_num=1
ssr based chi2 test:   chi2=1.4961  , p=0.2213  , df=1
likelihood ratio test: chi2=1.4946  , p=0.2215  , df=1
parameter F test:         F=1.4900  , p=0.2226  , df_denom=730, df_num=1
欄位[ Low

In [18]:
significant_col_list

['外資及陸資交易買口',
 '外資及陸資交易淨口',
 '自營商交易賣口',
 '外資及陸資未平倉賣口',
 '投信未平倉買口',
 '自營商未平倉賣口',
 '自營商未平倉淨口',
 '∆外資及陸資未平倉淨口',
 'Close',
 'Open',
 'last_day_return',
 'last_day_return_label',
 'day_return',
 'day_return_label_shift',
 'Date']

In [19]:
def get_significant_factor_df(shift_train_df,shift_test_df,significant_col_list):
    train_significant_df = shift_train_df[significant_col_list]
    test_significant_df = shift_test_df[significant_col_list]
    return (train_significant_df,test_significant_df)

In [20]:
train_significant_df,test_significant_df = get_significant_factor_df(shift_train_df,shift_test_df,significant_col_list)


In [21]:
train_significant_df.head(5)

,外資及陸資交易買口,外資及陸資交易淨口,自營商交易賣口,外資及陸資未平倉賣口,投信未平倉買口,自營商未平倉賣口,自營商未平倉淨口,∆外資及陸資未平倉淨口,Close,Open,last_day_return,last_day_return_label,day_return,day_return_label_shift,Date
1241,30207,-252,20185,25526,1772,7550,-945,-327,9250,9238,-31.0,0,12,0.0,2015-01-05
1242,29783,-7577,26943,27901,1611,5919,-265,-7817,9058,9144,-192.0,0,-86,1.0,2015-01-06
1243,22620,2176,16324,28812,1424,5724,-1194,1972,9044,9042,-14.0,0,2,1.0,2015-01-07
1244,33656,6978,22636,27432,1757,6820,769,7520,9256,9116,212.0,1,140,0.0,2015-01-08
1245,21934,-8333,21180,29005,1737,5015,1395,-8143,9230,9328,-26.0,0,-98,0.0,2015-01-09


## split independent variable and targets

In [22]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
scale = preprocessing.MinMaxScaler()

In [23]:
def split_targets(train_significant_df,test_significant_df):
    columns = ["Close","Open","Date","day_return_label_shift"]
    #train
    train_targets = train_significant_df["day_return_label_shift"]
    train_significant_df = train_significant_df.drop(columns = columns)
    #test
    test_targets = test_significant_df["day_return_label_shift"]
    test_significant_df = test_significant_df.drop(columns = columns)
    return (train_significant_df,train_targets,test_significant_df,test_targets)

In [24]:
train_data,train_targets,test_data,test_targets = split_targets(train_significant_df,test_significant_df)

In [25]:
test_data

,外資及陸資交易買口,外資及陸資交易淨口,自營商交易賣口,外資及陸資未平倉賣口,投信未平倉買口,自營商未平倉賣口,自營商未平倉淨口,∆外資及陸資未平倉淨口,last_day_return,last_day_return_label,day_return
1975,39224,3403,14741,23916,2054,13712,-2886,3403,75.0,1,79
1976,41530,-1211,21284,24877,2035,12011,-1837,-1211,80.0,1,40
1977,41637,988,15364,22797,1895,11554,-1312,988,49.0,1,6
1978,36624,-137,15675,21517,1818,11808,-1740,-213,21.0,1,20
1979,40314,-2023,12433,23925,1740,10689,1041,-2032,26.0,1,-2
...,...,...,...,...,...,...,...,...,...,...,...
2816,53519,1477,6012,49474,19452,3452,20970,1476,-41.0,0,-61
2817,81392,126,12605,51350,19187,4341,21080,138,-291.0,0,-85
2818,57672,395,9965,49247,19114,5973,19442,402,63.0,1,-129
2819,67540,-1407,12693,51170,19435,7005,18513,-1392,241.0,1,167


In [26]:
test_significant_df

,外資及陸資交易買口,外資及陸資交易淨口,自營商交易賣口,外資及陸資未平倉賣口,投信未平倉買口,自營商未平倉賣口,自營商未平倉淨口,∆外資及陸資未平倉淨口,Close,Open,last_day_return,last_day_return_label,day_return,day_return_label_shift,Date
1975,39224,3403,14741,23916,2054,13712,-2886,3403,10708,10629,75.0,1,79,1.0,2018-01-02
1976,41530,-1211,21284,24877,2035,12011,-1837,-1211,10788,10748,80.0,1,40,1.0,2018-01-03
1977,41637,988,15364,22797,1895,11554,-1312,988,10837,10831,49.0,1,6,1.0,2018-01-04
1978,36624,-137,15675,21517,1818,11808,-1740,-213,10858,10838,21.0,1,20,0.0,2018-01-05
1979,40314,-2023,12433,23925,1740,10689,1041,-2032,10884,10886,26.0,1,-2,0.0,2018-01-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2816,53519,1477,6012,49474,19452,3452,20970,1476,17254,17315,-41.0,0,-61,0.0,2021-06-18
2817,81392,126,12605,51350,19187,4341,21080,138,16963,17048,-291.0,0,-85,0.0,2021-06-21
2818,57672,395,9965,49247,19114,5973,19442,402,17026,17155,63.0,1,-129,1.0,2021-06-22
2819,67540,-1407,12693,51170,19435,7005,18513,-1392,17267,17100,241.0,1,167,1.0,2021-06-23


# result

In [27]:
import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error,mean_absolute_error

#XGboost
model_xgb = xgb.XGBRFClassifier(
     
    
)

model_xgb.fit(train_data, train_targets)
y_pred = model_xgb.predict(test_data)
accuracy_score=accuracy_score(test_targets, y_pred)
print(model_xgb,"正確率",accuracy_score)

[15:39:55] WARNING: /private/var/folders/ht/4yjsq_c55sq3zrbmw5nkz9lh0000gn/T/pip-install-29_8dmzl/xgboost_9784643f45fd4d87b2cc5a97906e6778/build/temp.macosx-11.0-arm64-3.8/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBRFClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bytree=1, gamma=0, gpu_id=-1, importance_type='gain',
                interaction_constraints='', max_delta_step=0, max_depth=6,
                min_child_weight=1, missing=nan, monotone_constraints='()',
                n_estimators=100, n_jobs=1, num_parallel_tree=100,
                objective='binary:logistic', random_state=0, reg_alpha=0,
                scale_pos_weight=1, tree_method='exact', validate_parameters=1,
                verbosity=None) 正確率 0.541371158392435


/opt/homebrew/Caskroom/miniforge/base/envs/stock_dl/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].



In [28]:
# test_significant_df = test_significant_df.iloc[:-1]
# test_significant_df

In [29]:
#手續費設兩點
def visualize_profit(test_shift_df,y_pred,fee = 2):
    #取出獲利欄位
    test_result_df = test_shift_df
    test_result_df["day_return_label_hat"] = y_pred
    
    #設計成,若看到昨天預測值為1,則今天獲利 = 今天收盤-昨天收盤
    test_result_df["day_return_label_hat_shift"] = test_result_df["day_return_label_hat"].shift(1)
    test_result_df["day_return_label_hat_shift"] = test_result_df["day_return_label_hat_shift"].replace(0,-1)
    #隔日沖
#     test_result_df["profit"] = test_result_df["day_return"]*test_result_df["day_return_label_hat_shift"]-fee
    #當沖
    test_result_df["profit"] = (test_result_df["Close"]-test_result_df["Open"])*test_result_df["day_return_label_hat_shift"]-fee

    test_result_df["sum_profit"] = test_result_df["profit"].cumsum()
    #繪圖
    layout = go.Layout(title='累計獲利圖', xaxis=dict(title='Date'), yaxis=dict(title='Point'))


    
    data = [
    {
        'x': test_result_df.Date,    
        'y': test_result_df.sum_profit,
        
    }
    ]
    
    test_result_df.iplot(data = data, layout = layout)

In [30]:
visualize_profit(test_significant_df,y_pred)